In [ ]:
import torch
import numpy as np
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import os
from PIL import Image
import PIL
import cv2 as cv
import tensorflow as tf
from keras.preprocessing import image
from keras.models import load_model
import keras.optimizers
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow as tf
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2

# Define class labels for the classification task
class_labels = [
    "combat",
    "destroyedbuilding",
    "empty",
    "fire",
    "humanitarianaid",
    "militaryvehicles",
]

# Define the size of the input images
IMG_SIZE = (160, 160)

# Check if CUDA is available, if not, default to CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)  # Print the device (CUDA if available, otherwise CPU)


In [ ]:
# Function Name: preprocess_and_display
# Input: image_path (str) - the path to the input image file
# Output: input_batch (torch.Tensor) - preprocessed image tensor
# Logic: This function preprocesses an image using a predefined transformation pipeline
#        and displays it using OpenCV if the 'display' flag is set to True.
# Example Call: input_batch = preprocess_and_display("path_to_image.jpg")

# Define the transformation pipeline for preprocessing images
transform = transforms.Compose(
    [
        transforms.Resize(IMG_SIZE),  # Resize the image to IMG_SIZE
        transforms.ToTensor(),  # Convert the image to a PyTorch tensor
        transforms.Normalize(  # Normalize the image using mean and std
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
    ]
)

def preprocess_and_display(image_path):
    img = Image.open(image_path)  # Open the image file using PIL
    input_tensor = transform(img)  # Apply the predefined transformation pipeline
    input_batch = input_tensor.unsqueeze(0)  # Add a batch dimension
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Check device availability
    input_batch = input_batch.to(device)  # Transfer the tensor to the appropriate device

    if display:
        numpy_img = input_tensor.permute(1, 2, 0).numpy()  # Convert tensor to NumPy array
        numpy_img = cv.cvtColor(numpy_img, cv.COLOR_BGR2RGB)  # Convert color space for display
        cv.imshow("Preprocessed Image", numpy_img)  # Display the preprocessed image
        cv.waitKey(200)  # Wait for a short duration
        cv.destroyAllWindows()  # Close the OpenCV windows

    return input_batch  # Return the preprocessed image tensor


In [ ]:

# Setting the directories for the training , test ,validation datasets and the model itself.

train_data_dir = r"C:\Users\moonr\Desktop\Bonus model training\Dataset\Train_Set"
test_data_dir = r"C:\Users\moonr\Desktop\Bonus model training\Dataset\Test_Set"
val_dir =  r"C:\Users\moonr\Desktop\Bonus model training\Dataset\Validation_Set"
model_dir = r"C:\Users\moonr\Desktop\Bonus model training\Model\modelv3.h5"


In [ ]:
# Function Name: create_image_data_generators
# Input: None
# Output: train_datagen (tf.keras.preprocessing.image.ImageDataGenerator),
#         test_datagen (tf.keras.preprocessing.image.ImageDataGenerator)
# Logic: This function creates image data generators for training and testing data.
#        It applies various data augmentation techniques to the images for better model generalization.
# Example Call: train_gen, test_gen = create_image_data_generators()

# Define the ImageDataGenerator for training data
train_datagen = ImageDataGenerator(
    rotation_range=5,  # Degree range for random rotations
    width_shift_range=0.1,  # Fraction of total width for horizontal shift
    height_shift_range=0.1,  # Fraction of total height for vertical shift
    shear_range=0.02,  # Shear intensity
    zoom_range=0.01,  # Range for random zoom
    horizontal_flip=1,  # Randomly flip inputs horizontally
    brightness_range=(0.1, 1.9),  # Range for adjusting brightness
    fill_mode="constant",  # Strategy for filling in newly created pixels
    cval=255,  # Value used for filling in newly created pixels
    preprocessing_function=preprocess_input,  # Function applied to each input
)

# Define the ImageDataGenerator for testing data
test_datagen = ImageDataGenerator(
    rotation_range=5,  # Degree range for random rotations
    width_shift_range=0.1,  # Fraction of total width for horizontal shift
    height_shift_range=0.1,  # Fraction of total height for vertical shift
    shear_range=0.02,  # Shear intensity
    zoom_range=0.01,  # Range for random zoom
    horizontal_flip=1,  # Randomly flip inputs horizontally
    brightness_range=(0.1, 1.9),  # Range for adjusting brightness
    fill_mode="constant",  # Strategy for filling in newly created pixels
    cval=255,  # Value used for filling in newly created pixels
    preprocessing_function=preprocess_input,  # Function applied to each input
)


In [ ]:
# Function Name: create_model_and_train
# Input: None
# Output: None
# Logic: This function creates a deep learning model for image classification using
#        transfer learning with the ResNet50V2 architecture. It then trains the model
#        using the provided training and testing data generators.
# Example Call: create_model_and_train()

# Define constants
NUM_CLASSES = 6  # Number of classes for classification
BATCH_SIZE = 32  # Batch size for training
EPOCHS = 40  # Number of epochs for training
LR = 0.0002  # Learning rate for the optimizer

TRAIN_DIR = train_data_dir  # Directory containing training images
TEST_DIR = test_data_dir  # Directory containing testing images

# Load the pre-trained ResNet50V2 model with weights from ImageNet
base_model = ResNet50V2(
    weights="imagenet", include_top=False, input_shape=IMG_SIZE + (3,)
)

# Freeze all layers in the base model to prevent their weights from being updated during training
for layer in base_model.layers:
    layer.trainable = False

# Create a Sequential model to append layers for transfer learning
model = Sequential()
model.add(base_model)  # Add the pre-trained base model
model.add(tf.keras.layers.GlobalAveragePooling2D())  # Global average pooling layer
model.add(Dropout(0.5))  # Add dropout after the pooling layer
model.add(
    Dense(NUM_CLASSES, activation="softmax", kernel_regularizer=l2(0.01))
)  # Add output layer with softmax activation for classification

# Compile the model with Adam optimizer and categorical crossentropy loss
optim = keras.optimizers.Adam(learning_rate=LR)
model.compile(optimizer=optim, loss="categorical_crossentropy", metrics=["accuracy"])

# Create data generators for training and testing data
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical"
)
test_generator = test_datagen.flow_from_directory(
    TEST_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical"
)

# Define model training callbacks
metric = "accuracy"
checkpoint = ModelCheckpoint(
    filepath=model_dir,
    monitor=metric,
    verbose=2,
    save_best_only=True,
    mode="max",
)
early_stopping = EarlyStopping(monitor="val_loss", patience=4)
callbacks = [checkpoint]

# Start model training and record the time taken for training
start = datetime.now()
model_history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=2,
)
duration = datetime.now() - start
print("Training completed in time: ", duration)


In [ ]:

# Plotting the values 

plt.plot(model_history.history["accuracy"])
plt.plot(model_history.history["val_accuracy"])
plt.title("CNN Model accuracy values")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Train", "Test"], loc="upper left")
plt.show()


In [ ]:
# Function Name: validate_model
# Input: model_dir (str) - directory containing the saved trained model,
#        val_dir (str) - directory containing validation data,
#        IMG_SIZE (tuple) - size of input images for validation
# Output: None
# Logic: This function validates the trained model using validation data.
#        It loads the model, iterates over the validation dataset, predicts
#        the classes of images, and calculates the classification accuracy
#        for each class. Finally, it prints the classification results.
# Example Call: validate_model(model_dir, val_dir, IMG_SIZE)

# Load the trained model from the saved directory
loaded_model = load_model(model_dir)

# Print the summary of the loaded model
print(loaded_model.summary())

# Define the class labels for classification
class_labels = [
    "combat",
    "destroyedbuilding",
    "empty",
    "fire",
    "humanitarianaid",
    "militaryvehicles",
]

# Initialize an empty list to store classification results
result_list = []

# Iterate over the directories containing validation data
for classes in os.listdir(val_dir):
    total_images = 0  # Initialize total number of images
    correct = 0  # Initialize number of correctly classified images

    # Iterate over images in each class directory
    for images in os.listdir(val_dir + "/" + classes):
        img_path = val_dir + "/" + classes + "/" + images  # Get the image path
        img = image.load_img(img_path, target_size=IMG_SIZE)  # Load the image
        img_array = image.img_to_array(img)  # Convert image to array
        img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions
        img_array = preprocess_input(img_array)  # Preprocess the input image
        probabilities = loaded_model.predict(img_array)  # Get class probabilities
        predicted_class_index = np.argmax(probabilities)  # Get the predicted class index
        pred = class_labels[predicted_class_index]  # Get the predicted class label

        # Check if the prediction matches the true class label
        if classes == pred:
            correct += 1  # Increment correct count
        total_images += 1  # Increment total images count

    # Append the result for the current class to the result_list
    result_list.append(
        str(
            str(correct)
            + "/"
            + str(total_images)
            + "    "
            + str(round((correct / total_images) * 100))
            + "%   --> "
            + classes
        )
    )

# Print the classification results for each class
for i in result_list:
    print(i)
